### CoCoMac Thalamic Projections

general strategy:
1. find all brainsites in CoCoMac that are part of Thalamus
2. find all injections in those brainsites
3. find all labelledsites in those brainsites

The combined result of 2 and 3 indicates which publications deal with Thalamus.

## 1. find all brainsites in CoCoMac that are part of Thalamus

This isn't so trivial, since CoCoMac doesn't use a unified atlas. All brainsites in CoCoMac are losely connected through IntermapRelations. There is one coarse atlas in CoCoMac to which many brainsites are linked, it is called General Map (GM). Let's find all regions that are subareas of area GM-Tha. Spoiler: 1138 hits.

In [1]:
import requests

# Load all brainsite-ids that have the spatial relation 'smaller than' thalamus,
# where the General Map brainsite 'GM-Thal' represents the thalamus.

'''
Type of data to search for	:	brain site
Apply a constraint	:	Yes
Property to limit search by	:	◄ vector properties of BrainMaps_BrainSites
Use operator	:	IN (subquery)
. . Table.field that links to BrainMaps_BrainSites	:	QRT_MappingPaths.ID_BrainSite_B
. . Apply a constraint	:	Yes
. . Property to limit search by	:	BrainMaps_BrainSites.BrainSite
. . Use operator	:	equals
. . Use value	:	GM-Tha
. . Add another constraint	:	Yes, combine using AND
. . Property to limit search by	:	QRT_MappingPaths.RelationCode
. . Use operator	:	equals
. . Use value	:	L
. . Add another constraint	:	No
Add another constraint	:	No
'''
r = requests.get('http://cocomac.g-node.org/services/search_wizard.php?T=BrainMaps_BrainSites&x0=WHERE&L0=&op0=IN&R0T=QRT_MappingPaths.ID_BrainSite_B&R0x0=WHERE&R0L0=^ID_BrainSite_A.BrainSite&R0op0=eq&R0R0=GM-Tha&R0x1=AND&R0L1=.RelationCode&R0op1=eq&R0R1=L&R0x2=&x1=&limit=2000&page=1&format=keys')
brainsiteIds = r.json()['resultKeys']
len(brainsiteIds)

1138

# 2. find all injections in those brainsites
After some juggling with the 'CoCoMac Search Wizard' and 'Direct SQL Query', zero injections turn out to be in any of the 1138 thalamic regions.

In [2]:
# next, find injection-ids that contain an injection site in any of these regions.
query = {
  "sql":"SELECT T0.ID,T0.ID_Method,T0.ID_BrainSite,T0.SiteType FROM Injections AS T0 LEFT JOIN BrainMaps_BrainSites AS T1 ON T0.ID_BrainSite=T1.ID WHERE T1.ID IN ($R0)",
  "params":{
      'R0':','.join(brainsiteIds[0:100])
  }
}

r = requests.post('http://cocomac.g-node.org/services/custom_sql_query.php?sql=SELECT%20T0.ID%2CT0.ID_Method%2CT0.ID_BrainSite%2CT0.SiteType%20FROM%20Injections%20AS%20T0%20LEFT%20JOIN%20BrainMaps_BrainSites%20AS%20T1%20ON%20T0.ID_BrainSite%3DT1.ID%20WHERE%20T1.ID%20IN%20(%24R0)&R0=3%2C138%2C140%2C146%2C147%2C148%2C149%2C151%2C153%2C155%2C156%2C160%2C161%2C163%2C164%2C167%2C171%2C172%2C173%2C174%2C175%2C176%2C177%2C178%2C179%2C180%2C181%2C182%2C187%2C188%2C189%2C190%2C191%2C192%2C193%2C304%2C330%2C336%2C337%2C338%2C339%2C340%2C341%2C343%2C345%2C380%2C559%2C648%2C649%2C652%2C654%2C662%2C663%2C664%2C665%2C666%2C667%2C668%2C669%2C670%2C671%2C814%2C815%2C816%2C818%2C819%2C820%2C821%2C822%2C824%2C825%2C826%2C828%2C830%2C831%2C833%2C836%2C837%2C838%2C903%2C904%2C905%2C906%2C907%2C913%2C914%2C915%2C916%2C922%2C923%2C965%2C966%2C967%2C968%2C969%2C970%2C971%2C972%2C973%2C974%2C975&format=json_rpc&truncate=1000',
                 data={'R0':','.join(brainsiteIds)})
print(r.json())

# it looks that there are no injections in thalamus whatsoever

{'result': {'keys': ['ID'], 'fields': ['ID', 'ID_Method', 'ID_BrainSite', 'SiteType'], 'data': []}}


# 3. find all injections in those brainsites
Repeat the exercise for LabelledSites_Data. Again zero data in any of the 1138 thalamic regions.

In [3]:
# next, find labelledsite-ids that contain an injection site in any of these regions.
query = {
  "sql":"SELECT T0.ID,T0.ID_Description,T0.ID_BrainSite,T0.SiteType,T0.Density FROM LabelledSites_Data AS T0 LEFT JOIN BrainMaps_BrainSites AS T1 ON T0.ID_BrainSite=T1.ID WHERE T1.ID IN ($R0)",
  "params":{
      'R0':','.join(brainsiteIds[0:100])
  }
}

    
r = requests.post('http://cocomac.g-node.org/services/custom_sql_query.php?sql=SELECT%20T0.ID%20FROM%20LabelledSites_Data%20AS%20T0%20LEFT%20JOIN%20BrainMaps_BrainSites%20AS%20T1%20ON%20T0.ID_BrainSite%3DT1.ID%20WHERE%20T1.ID%3D%24R0&format=json_rpc&truncate=1000',
                 data={'R0':','.join(brainsiteIds)})
print(r.json())

# it looks that there are no labelled sites in thalamus either

{'result': {'keys': ['ID'], 'fields': ['ID'], 'data': []}}


# Switch strategy: find papers with thalamus keyword

Let's then look for the word thalamus in the abstract/title

There are 87 journal publications in cocomac with the word 'thalam' in the abstract:
http://cocomac.g-node.org/services/search_wizard.php?T=Literature_JournalArticles&x0=WHERE&L0=%5EID_Literature.Abstract&op0=*x*&R0=thalam&x1=&limit=100&page=1&format=dhtml

There are 87 journal publications in cocomac with the word 'thalam' in the title:
http://cocomac.g-node.org/services/search_wizard.php?T=Literature_JournalArticles&x0=WHERE&L0=%5EID_Literature.Title&op0=*x*&R0=thalam&x1=&limit=100&page=1&format=dhtml

The second hit is this one:
https://pubmed.ncbi.nlm.nih.gov/6768425/
based on the abstract,it should have labelled sites in the Thalamus.

In the text it says: 
```
Thalamic label was found almost exclu-
sively in non-specific nuclei with the exception of the magnocellular part of the ventral
anterior (VAmc) nucleus which was labelled after the injections centred on the lateral,
and possibly the lateral basal nuclei. The majority of the remaining labelled cells were
found in the following midline and intralaminar thalamic nuclei: nucleus paraventri-
cularis (Pa, Pac), nucleus centralis latocellutaris (Clc), nucleus centralis densocellularis
(Cdc), nucleus centralis superior (Cs), nucleus centralis inferior (Cif), nucleus centralis
361
intermedialis (Cim), nucleus rotundis (Ro), nucleus reuniens (Re), nucleus paracen-
tralis (Pcn), nucleus subfascicularis (Sf.pc and Sf.mc) and nucleus centrum medianum
(Cn.Md). A
```

Next, looking at the CoCoMac representation of this experiment, there are many labelled sites in the Thalamus
http://cocomac.g-node.org/services/search_wizard.php?T=Literature_JournalArticles&x0=WHERE&L0=%5EID_Literature.Title&op0=*x*&R0=thalam&x1=&limit=100&page=1&format=dhtml

So what is going wrong is that these brainsites are not linked the GM-Tha brainsite and were therefore missed in the above analysis.
        

# Next steps
device a strategy for a uniform atlas for the CoCoMac database. We could start with the very coarse General Map, and make sure that all brain regions in CoCoMac have a relation to it.